In [ ]:
!pip install segmentation-models dagshub mlflow

In [ ]:
import os
import cv2
from google.colab import userdata

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import dagshub
import mlflow
import pickle

from PIL import Image
import albumentations as A
import shutil

import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import gc

# Set the SM_FRAMEWORK environment variable before importing segmentation_models
os.environ['SM_FRAMEWORK'] = 'tf.keras'

import segmentation_models as sm

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Récupère automatiquement le secret
dagshub_token = userdata.get('DAGSHUB_TOKEN')

# Initialisation Dagshub
dagshub.auth.add_app_token(dagshub_token)

# Connecter MLflow à Dagshub
dagshub.init(repo_owner='fabiencappelli', repo_name='Projet_08', mlflow=True)

from matplotlib import rcParams
import matplotlib.font_manager as fm
font_path = os.path.expanduser("/content/drive/MyDrive/Colab Notebooks/fonts/Exo2-VariableFont_wght.ttf")
fm.fontManager.addfont(font_path)

# Définir la police globale avec le nom de la police
rcParams["font.family"] = "Exo 2"
# deux couleurs pertinentes pour aller avec la présentation
bleuclair = (0.15, 0.55, 0.82)
couleur_complementaire = (1 - bleuclair[0], 1 - bleuclair[1], 1 - bleuclair[2])
bleufonce = "#073642"

In [ ]:
imgPrezPath = '/content/drive/MyDrive/Colab Notebooks/Projet 8/img'
masksSourceFolder = '/content/drive/MyDrive/Colab Notebooks/Projet 8/P8_Cityscapes_gtFine_trainvaltest/gtFine8'
imagesSourceFolder = '/content/drive/MyDrive/Colab Notebooks/Projet 8/P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit'
masksFolder = '/content/images'
imagesFolder = '/content/masks'

In [ ]:
sources = [masksSourceFolder, imagesSourceFolder]
destinations = [masksFolder, imagesFolder]
splits = ['train', 'val']

In [ ]:
for split in splits:
  for source, destination in zip(sources, destinations):
    src_dir = os.path.join(source, split)
    dst_dir = os.path.join(destination, split)
    os.makedirs(destination, exist_ok=True)
    os.makedirs(dst_dir, exist_ok=True)
    for fichier in os.listdir(src_dir):
      if fichier.endswith("labelIds.png") or fichier.endswith("leftImg8bit.png"):
        shutil.copyfile(os.path.join(src_dir, fichier), os.path.join(dst_dir, fichier))

In [ ]:
BACKBONE = 'efficientnetb0'

In [ ]:
preprocess_input = sm.get_preprocessing(BACKBONE)

# Dataloader

In [ ]:
dicoclasses = {0:'void',
               1:'flat',
               2:'construction',
               3:'object',
               4:'nature',
               5:'sky',
               6:'human',
               7:'vehicle',
              }

def to_onehot(mask, num_classes=8):
    return tf.one_hot(mask, num_classes).numpy().astype(np.float32)

# adapted from https://github.com/qubvel/segmentation_models/blob/master/examples/multiclass%20segmentation%20(camvid).ipynb
class Dataloader(tf.keras.utils.Sequence):
    """Load data from dataset and form batches

    Args:
        data_folder: folder where is data.
        batch_size: Integet number of images in batch.
        transform: albumentations.Compose.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """

    def __init__(self, data_folder, batch_size=1, transform=None, shuffle=False):
        self.data_folder = data_folder
        if data_folder == X_train_dir:
            self.mask_folder = y_train_dir
        elif data_folder == X_val_dir:
            self.mask_folder = y_val_dir
        elif data_folder == X_test_dir:
            self.mask_folder = y_test_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.transform = transform
        self.indexes = list(set([path[:path.rfind('_')] for path in os.listdir(data_folder) if path.endswith(".png") ]))
        self.mask_indexes = [os.path.join(self.mask_folder, path) for path in self.indexes]
        self.indexes = [os.path.join(data_folder, path) for path in self.indexes]

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        images = []
        masks = []
        for j in range(start, stop):
            root_file = self.indexes[j]
            mask_file = self.mask_indexes[j]
            image_file = root_file + "_leftImg8bit.png"
            mask_file = mask_file + "_gtFine_labelIds.png"
            # on convertit les images en array numpy
            image = np.array(Image.open(image_file))
            mask = np.array(Image.open(mask_file))
            # Appliquer la transformation si elle est demandée
            if self.transform is not None:
                transformed = self.transform(image=image, mask=mask)
                image = transformed["image"]
                mask = transformed["mask"]
            # c'est à ce moment qu'on encode en one-hot
            mask = to_onehot(mask)
            # on les ajoute à leurs listes respectives
            images.append(image)
            masks.append(mask)
        # transpose list of lists
        # batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        image_batch = np.stack(images, axis=0)
        mask_batch = np.stack(masks, axis=0)
        # print("DEBUG | images", image_batch.shape, image_batch.dtype)
        # print("DEBUG | masks ", mask_batch.shape, mask_batch.dtype)
        # print("DEBUG | unique mask values", np.unique(mask_batch))
        return image_batch, mask_batch

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
        # return int(np.ceil(len(self.indexes) / self.batch_size))


    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
          # On mélange les paires image/mask ENSEMBLE
          combined = list(zip(self.indexes, self.mask_indexes))
          np.random.shuffle(combined)
          self.indexes, self.mask_indexes = zip(*combined)
          # zip(*combined) retourne des tuples, donc si tu veux des listes :
          self.indexes = list(self.indexes)
          self.mask_indexes = list(self.mask_indexes)

train_transform = A.Compose([
    # 1. Cropping / Resize
    A.Resize(256, 512),

    # 2. Basic Geometric (invariances basiques)
    # A.HorizontalFlip(p=0.5),
    # Pas de flip vertical, pas de symétrie carrée (sauf imagerie satellite)

    # 3. Dropout/Occlusion (pour la robustesse aux obstacles)
    # A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.2),

    # 4. Color/Channel dropout (si tu veux vraiment rendre le modèle insensible à la couleur)
    # A.ToGray(p=0.1),
    # A.ChannelDropout(p=0.1),

    # 5. Affine transformations
    # A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.05, rotate_limit=5, p=0.25),

    # 6. Domain-Specific (effets météo, soleil, etc.)
    # A.RandomSunFlare(p=0.1),
    # A.RandomShadow(p=0.1),
    # A.RandomFog(p=0.05),
    # A.RandomRain(p=0.05),
    # A.RandomSnow(p=0.05),
    # Autres effets spécifiques :
    A.RandomBrightnessContrast(p=0.2),
    # A.GaussNoise(p=0.2),

    # 7. Normalization (toujours à la fin)
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
    A.Lambda(image=preprocess_input)
])


val_transform = A.Compose([
    A.Resize(256, 512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
    A.Lambda(image=preprocess_input)
])

# Callbacks

In [ ]:
earlystop_cb = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)

lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

class PerClassMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_loader, num_classes=8):
        # on initie d'abord le parent pour éviter les mauvaises surprises
        super().__init__()
        self.val_loader = val_loader
        self.num_classes = num_classes
        # on charge le dico des classes
        self.dicoclasses = dicoclasses or {i: f"class_{i}" for i in range(num_classes)}
        # on stocke les scores de chaque epoch dans un df
        self.df_scores = pd.DataFrame()

    def on_epoch_end(self, epoch, logs=None):
        # à chaque fin d'époque on calcule les métriques customs
        # d'abord on recueille les prédictions et la vérité
        all_preds = []
        all_trues = []
        for i in range(len(self.val_loader)):
            imgs, masks = self.val_loader[i]
            preds = self.model.predict(imgs, verbose=0)
            all_preds.append(preds)
            all_trues.append(masks)
        all_preds = np.concatenate(all_preds, axis=0)
        all_trues = np.concatenate(all_trues, axis=0)
        # on extrait la plus haute probabilité
        y_pred = np.argmax(all_preds, axis=-1)           # (n_samples, H, W)
        # on convertit aussi les ground truths en indices
        if all_trues.ndim == 4 and all_trues.shape[-1] > 1:
            y_true = np.argmax(all_trues, axis=-1)       # (n_samples, H, W)
        else:
            y_true = all_trues

        dice_scores = []
        iou_scores = []
        for c in range(self.num_classes):
            y_true_c = (y_true == c).astype(np.int32)
            y_pred_c = (y_pred == c).astype(np.int32)
            intersection = (y_true_c * y_pred_c).sum()
            union = y_true_c.sum() + y_pred_c.sum()
            dice = (2. * intersection) / (union + 1e-6)
            dice_scores.append(dice)

            union_iou = y_true_c.sum() + y_pred_c.sum() - intersection
            iou = (intersection) / (union_iou + 1e-6)
            iou_scores.append(iou)

        # ajouts au dataframe de résultats
        row = {'epoch': epoch+1}
        for c, (d, iou) in enumerate(zip(dice_scores, iou_scores)):
            label = self.dicoclasses.get(c, f"class_{c}")
            row[f"dice_{label}"] = d
            row[f"iou_{label}"] = iou
        row['dice_mean'] = np.mean(dice_scores)
        row['iou_mean'] = np.mean(iou_scores)

        self.df_scores = pd.concat([self.df_scores, pd.DataFrame([row])], ignore_index=True)


        print(f"\n=== Époque {epoch + 1} ===")
        for c, (d, iou) in enumerate(zip(dice_scores, iou_scores)):
            className = dicoclasses.get(c, f"class_{c}")
            print(f"{className} (classe {c}): Dice={d:.4f} | IoU={iou:.4f}")
        print(f"--- Dice moyen: {np.mean(dice_scores):.4f} | IoU moyen: {np.mean(iou_scores):.4f} ---\n")



# Fit sur dataset complet

In [ ]:
DECODER = 'Linknet'

In [ ]:
from sklearn.model_selection import train_test_split
import glob
import time

In [ ]:
X_train_dir = os.path.join(imagesFolder, 'train')
y_train_dir = os.path.join(masksFolder, 'train')

X_val_dir = os.path.join(imagesFolder, 'val')
y_val_dir = os.path.join(masksFolder, 'val')

train_imgs = sorted(glob.glob(os.path.join(X_train_dir, "*_leftImg8bit.png")))
train_masks = sorted(glob.glob(os.path.join(y_train_dir, "*_gtFine_labelIds.png")))

assert len(train_imgs) == len(train_masks), "Mismatch images/masks"

# Split 90% train / 10% test (val déjà à part)
train_imgs_new, test_imgs_new, train_masks_new, test_masks_new = train_test_split(
    train_imgs, train_masks, test_size=0.1, random_state=34
)

os.makedirs(os.path.join(imagesFolder, 'test'), exist_ok=True)
os.makedirs(os.path.join(masksFolder, 'test'), exist_ok=True)

X_test_dir = os.path.join(imagesFolder, 'test')
y_test_dir = os.path.join(masksFolder, 'test')

for img, mask in zip(test_imgs_new, test_masks_new):
    shutil.move(img, X_test_dir)
    shutil.move(mask, y_test_dir)

In [ ]:
batch_size = 8
learning_rate = 1e-3

train_loader = Dataloader(
    data_folder=X_train_dir,
    batch_size=batch_size,
    transform=train_transform,
    shuffle=True
)
val_loader = Dataloader(
    data_folder=X_val_dir,
    batch_size=batch_size,
    transform=val_transform,
    shuffle=False
)
test_loader = Dataloader(
    data_folder=X_test_dir,
    batch_size=4,
    transform=val_transform,
    shuffle=False
)
per_class_metrics_cb = PerClassMetricsCallback(val_loader=val_loader, num_classes=8)
optimizer = Adam(learning_rate=learning_rate)
model = sm.Linknet(BACKBONE, encoder_weights='imagenet', classes=8, activation='softmax')
model.compile(
    optimizer,
    loss='categorical_crossentropy',
    metrics=[sm.metrics.iou_score],
)
EXPERIMENT = 'FULL_' + BACKBONE + '_' + DECODER + '_BS_' + str(batch_size) + '_LR_' + str(learning_rate)
mlflow.set_experiment(EXPERIMENT)
with mlflow.start_run():
    history = model.fit(
        train_loader,
        validation_data=val_loader,
        epochs=50,
        callbacks=[earlystop_cb, per_class_metrics_cb, lr_callback],
        verbose=1
    )
    # Log des hyperparamètres :
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("lr", learning_rate)
    # Log des métriques epoch par epoch :
    for epoch, (loss, val_loss, iou, val_iou) in enumerate(zip(
        history.history['loss'],
        history.history['val_loss'],
        history.history['iou_score'],
        history.history['val_iou_score']
    )):
        mlflow.log_metric("loss", loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("iou_score", iou, step=epoch)
        mlflow.log_metric("val_iou_score", val_iou, step=epoch)
    # Sauvegarde de l'objet history complet :
    with open("history.pkl", "wb") as f:
        pickle.dump(history.history, f)
    mlflow.log_artifact("history.pkl")
    # Instancie la métrique IoU fournie par segmentation_models
    iou_metric = sm.metrics.IOUScore()

    start_time = time.time()
    n_samples = 0

    all_preds, all_trues = [], []

    for imgs, masks in test_loader:
        preds = model.predict(imgs)
        all_preds.append(preds)
        all_trues.append(masks)
        n_samples += imgs.shape[0]

    y_pred = tf.concat(all_preds, axis=0)
    y_true = tf.concat(all_trues, axis=0)

    elapsed_time = time.time() - start_time
    avg_inference_time = elapsed_time / n_samples

    mean_iou_test = sm.metrics.iou_score(y_true, y_pred).numpy()

    mlflow.log_metric("test_avg_inference_time", avg_inference_time)
    mlflow.log_metric("test_mean_iou", mean_iou_test)



In [ ]:
plt.figure()

plt.gca().xaxis.set_major_locator(mticker.MaxNLocator(integer=True))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.plot(history.history['iou_score'], label='train IoU_score')
plt.plot(history.history['val_iou_score'], label='val IoU_score')
plt.axhline(mean_iou_test, color='purple', linestyle='--', label=f'test IoU={mean_iou_test:.3f}')

plt.legend()
plt.title(EXPERIMENT + ' avec augmentation')
plt.xlabel('Époques')
plt.ylabel('Valeurs')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, EXPERIMENT +'.svg'),format="svg",bbox_inches="tight",pad_inches=0.1,)

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/Projet 8/' + EXPERIMENT + '.keras')

In [ ]:
test_loader = Dataloader(
    data_folder=X_test_dir,
    batch_size=1,           # Pour la démo, plus lisible
    transform=val_transform,
    shuffle=False
)

In [ ]:
images, masks = test_loader[1]
image = images[0]
true_mask = masks[0]

In [ ]:
# on doit rajouter une dimension à l'image au début pour qu'elle ait la bonne forme pour entrer dans le modèle (batch)
pred = model.predict(np.expand_dims(image, 0))
# on réduit le mask de one hot à une image
pred_mask = np.argmax(pred[0], axis=-1)
image_file = test_loader.indexes[1] + "_leftImg8bit.png"
original_image = np.array(Image.open(image_file))
original_shape = original_image.shape[:2][::-1] # (width, height)
pred_mask_resized = np.array(
    Image.fromarray(pred_mask.astype(np.uint8)).resize(
        original_shape, resample=Image.NEAREST
    )
)

# Affichage
fig, axs = plt.subplots(1, 3, figsize=(16, 5))
axs[0].imshow(original_image)
axs[0].set_title("Image d'origine")
axs[0].axis("off")
axs[1].imshow(pred_mask_resized, cmap="tab20", vmin=0, vmax=7)
axs[1].set_title("Mask prédit (taille d'origine)")
axs[1].axis("off")
axs[2].imshow(image)
axs[2].set_title("Image prétraitée (entrée modèle)")
axs[2].axis("off")
plt.tight_layout()
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "trio_linknet_pred.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)